Test running a GraphComponent with one of its nodes as a ProxyNode

In [ ]:
#|default_exp concurrent.remote.test_ProxyNode_2

In [ ]:
#|export
import asyncio, os
from fbdev.dev_utils import is_in_repl
from fbdev.graph.net import NodeSpec
from fbdev.concurrent.remote import ProxyNode
from fbdev.comp.port import  PortSpecCollection, PortSpec, PortType
from fbdev.graph.graph_spec import GraphSpec
from fbdev.graph import GraphComponentFactory
from fbdev.concurrent.remote import ProxyNode, Node

from _test_ProxyNode_1 import FooComponent1, FooComponent2

In [ ]:
#|export
graph = GraphSpec(PortSpecCollection(
    PortSpec(PortType.INPUT, 'inp'),
    PortSpec(PortType.OUTPUT, 'out'),
))

node_foo_component1 = graph.add_node(FooComponent1, node_type=ProxyNode)
node_foo_component2 = graph.add_node(FooComponent2)

graph.ports.input.inp >> node_foo_component1.ports.input.inp
node_foo_component1.ports.output.out >> node_foo_component2.ports.input.inp
node_foo_component2.ports.output.out >> graph.ports.output.out

graph.display_mermaid(hide_unconnected_ports=True)

GraphComponent = GraphComponentFactory.create_component(graph)
GraphComponent.set_module()

In [ ]:
#|export
async def main():
    node_spec = NodeSpec(GraphComponent)
    proxy_node = Node(node_spec, parent_net=None)
    
    await proxy_node.task_manager.exec_coros(
        proxy_node.start(),
        proxy_node.ports.input.inp.put_value('Hello child!'),
        proxy_node.stop(),
        sequentially=True
    )
    
if not is_in_repl():
    os.chdir(os.path.dirname(os.path.abspath(__file__)))
    asyncio.run(main())

In [ ]:
await main()

Exception chain 1:
    Exception 1 (RecursionError):
        Source: remote_node_task_manager
        Message: maximum recursion depth exceeded
        Traceback:
            RecursionError: maximum recursion depth exceeded
    Exception 2 (RemoteNodeError):
        Source: <fbdev.concurrent.remote.ProxyNode object>
        Message: 
        Traceback:
            RecursionError: maximum recursion depth exceeded
            
            The above exception was the direct cause of the following exception:
            
            Traceback (most recent call last):
              File "/Users/lukastk/proj_dev/fbdev/fbdev/concurrent/remote.py", line 513, in _submit_exception_from_remote
                raise RemoteNodeError() from exceptions[0]
            fbdev.concurrent.remote.RemoteNodeError
    Exception 3 (NodeError):
        Source: <GraphComponent object>
        Message: 
        Traceback:
            RecursionError: maximum recursion depth exceeded
            
            The a

RecursionError: maximum recursion depth exceeded